## Prepare Dependencies

In [ ]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph

In [ ]:
!pip install datasets
from datasets import Dataset
!pip install ragas
from ragas import evaluate
from ragas.metrics import (
    faithfulness,
    answer_relevancy
)

In [ ]:
!pip install -U langchain-openai
!pip install openai
!pip install tiktoken
!pip install -U qdrant-client
!pip install langchain_experimental
!pip install "langchain[docarray]"

In [ ]:
from openai import OpenAI
import httpx

#Set the langchain
import os
os.environ["OPENAI_API_BASE"] = 'https://api.xty.app/v1'
os.environ["OPENAI_API_KEY"] = 'sk-xARsvcz0Lu8vU8LzA8F845704f1d4c0180087e19137e0d2e'

In [ ]:
from langchain.document_loaders import CSVLoader
from langchain_openai import OpenAIEmbeddings
from qdrant_client import QdrantClient
from langchain_community.vectorstores import Qdrant
from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

## Data Prepare

In [ ]:
import pandas as pd
c1 = pd.read_csv("01-papers.csv")
c2 = pd.read_csv("02-papers.csv")
c3 = pd.read_csv("03-papers.csv")
c4 = pd.read_csv("04-papers.csv")
c5 = pd.read_csv("05-papers.csv")
c6 = pd.read_csv("06-papers.csv")
c7 = pd.read_csv("07-papers.csv")
c8 = pd.read_csv("08-papers.csv")
c9 = pd.read_csv("09-papers.csv")
c10 = pd.read_csv("10-papers.csv")

In [ ]:
papers = pd.concat([c1,c2,c3,c4,c5,c6,c7,c8,c9,c10], axis=0)

In [ ]:
papers['info'] = papers['author'].map(str) + papers['year'].map(str) + papers['title'].map(str)
data = papers.drop(columns = ['author','year','title'])
data.to_csv('001_DATA.csv')

In [ ]:
pd.set_option('max_colwidth', 700)
data

In [ ]:
loader = CSVLoader(file_path="001_DATA.csv", source_column="info")
data = loader.load()

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print('done.')

In [ ]:
# Take a look of your documents
print (f'Now you have {len(chunks)} documents')
print(chunks[90].page_content)

In [ ]:
#Set an embedding model
embeddings = OpenAIEmbeddings(model = 'text-embedding-3-small')

In [ ]:
#Build a vector store
story = Qdrant.from_documents(
    chunks,
    embeddings,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

In [ ]:
#Build a retriever
retriever = story.as_retriever(search_type="mmr") # Maximal marginal relevance

In [ ]:
documents = chunks

## Prompt

In [ ]:
# Define LLM
llm = ChatOpenAI(model_name="gpt-4", temperature=0, max_tokens = 150)

# Define prompt template
template = """You are an expert for the COVID-19 fact-checking tasks.
Based on pieces of retrieved context to detect the claim is true or false.
You will have to give me the title and author of the context you refered in one sentence.
If you don't know the answer, just say that you don't know.
Keep the answer concise.
Claim: {question}
Context: {context}
Answer:
"""

prompt = ChatPromptTemplate.from_template(template)

In [ ]:
retriever = retriever

In [ ]:
# Setup RAG pipeline
#retriever = retriever11
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
#compression_retriever_reordered

In [ ]:
import pandas as pd
claim = pd.read_csv('500_g.csv')
claim

## Baseline

In [ ]:
# Define baseline LLM
llm_b = ChatOpenAI(model_name="gpt-4", temperature=0,max_tokens = 300)

# Define prompt template
template_b = """You are an expert for the COVID-19 fact-checking tasks.
Please check the claim is true or false.
If you don't know the answer, just say that you don't know.
Use one sentences maximum and keep the answer concise.
Claim: {question}
Answer:
"""

prompt_b = ChatPromptTemplate.from_template(template_b)

In [ ]:
base_line = pd.read_csv('claim_e.csv')
base_line

In [ ]:
rag_chain_b = (
    {"question": RunnablePassthrough()}
    | prompt_b
    | llm_b
    | StrOutputParser()
)

In [ ]:
questions = base_line['question']
ground_truths = base_line['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    answers.append(rag_chain_b.invoke(query))

# To dict
data_b = {
    "question": questions,
    "answer": answers,
    "ground_truths": ground_truths
}

In [ ]:
# Convert dict to dataset
dataset_b = Dataset.from_dict(data_b)

tail = dataset_b.to_pandas()
tail.to_csv('gpt-4_b_e.csv')

In [ ]:
tail

## Naive RAG

In [ ]:
import warnings
warnings.filterwarnings('ignore', category=FutureWarning)

In [ ]:
questions = claim['question']
ground_truths = claim['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    answers.append(rag_chain.invoke(query))
    docs = retriever.invoke(query)
    contexts.append([docs.page_content for docs in retriever.invoke(query)])
    #contexts.append(docs[0].page_content)

# To dict
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truths": ground_truths
}

# Convert dict to dataset
dataset = Dataset.from_dict(data)

result = evaluate(
    dataset = dataset,
    metrics=[
        answer_relevancy],
)

res = result.to_pandas()

In [ ]:
res = dataset.to_pandas()
res

In [ ]:
res.to_csv('naive_rag_e.csv')

## LOTR-RAG

In [ ]:
embedding_2 = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

story_2 = Qdrant.from_documents(
    chunks,
    embedding_2,
    location=":memory:",  # Local mode with in-memory storage only
    collection_name="my_documents",
)

#Build a retriever
retriever_2 = story_2.as_retriever(search_type="mmr") # Maximal marginal relevance

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import DocumentCompressorPipeline
from langchain.retrievers.merger_retriever import MergerRetriever

lotr_full = MergerRetriever(retrievers=[retriever,retriever_2])

In [ ]:
#Test the Mergered Retrievers.
docs = lotr_full.invoke(
    "What is COVID-19?")
print(len(docs))

In [ ]:
# Build the Filter and Reordering. Please Attention to the cost.
from langchain.document_transformers import (
     EmbeddingsClusteringFilter,
     EmbeddingsRedundantFilter,
 )
filter = EmbeddingsRedundantFilter(embeddings=embeddings)
pipeline = DocumentCompressorPipeline(transformers=[filter])
compression_retriever = ContextualCompressionRetriever(base_compressor=pipeline,
                                                       base_retriever=lotr_full)
docs = compression_retriever.invoke(
    "What is COVID-19?")
print(len(docs))

In [ ]:
from re import search
from langchain.document_transformers import LongContextReorder
reordering = LongContextReorder()
pipeline = DocumentCompressorPipeline(transformers=[filter, reordering])
compression_retriever_reordered = ContextualCompressionRetriever(
     base_compressor=pipeline, base_retriever=lotr_full,search_kwargs={"k": 3, "include_metadata": True}
 )
docs = compression_retriever_reordered.get_relevant_documents(
    "What is COVID-19?")
print(len(docs))

In [ ]:
# Setup Naive RAG chain

rag_chain_lotr = (
    {"context": compression_retriever_reordered, "questions": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
questions = claim['question']
ground_truths = claim['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    answers.append(rag_chain_lotr.invoke(query))
    docs = compression_retriever_reordered.invoke(query)
    contexts.append([docs.page_content for docs in compression_retriever_reordered.invoke(query)])
    #contexts.append(docs[0].page_content)

In [ ]:
# To dict
dict_lotr = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "path_id": path_id
}

res_lotr = pd.DataFrame(dict_lotr)
res_lotr

In [ ]:
res_lotr.to_csv('LOTR-RAG-Res-E.csv')

In [ ]:
# Combine the results
res_lotr = pd.read_csv('LOTR-RAG-Res-E.csv')
# 给DataFrame应用函数
l_rag=res_lotr.groupby(['path_id'])['question'].apply(list).to_frame()
l_rag['question']=l_rag['question'].apply(lambda x:str(x).replace('[','').replace(']',''))
o_rag=res_n.groupby(['path_id'])['answer'].apply(list).to_frame()
o_rag['question']=o_rag['answer'].apply(lambda x:str(x).replace('[','').replace(']',''))
o_rag

In [ ]:
o_rag.to_csv('C-LOTR-RAG-Res-E.csv')

In [ ]:
df = pd.read_csv('SP.csv')
df

## Corrective RAG (CRAG)

In [ ]:
# Implement the Router

import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser
vectorstore = story
llm_w = ChatOpenAI(model="gpt-4", temperature=0)
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are an expert at routing a
    user question to a vectorstore or web search. Use the vectorstore for questions on COVID-19.
    You do not need to be stringent with the keywords.
    in the question related to these topics. Otherwise, use web-search. Give a binary choice 'web_search'
    or 'vectorstore' based on the question.
    Return the a choice with a single key 'datasource' and no premable or explaination.
    Question to route: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question"],
)
start = time.time()
question_router = prompt | llm_w | JsonOutputParser()
#
question = "High levels of a cytokine called interleukin-6 (IL-6) are not associated with respiratory failure and death in COVID-19 patients."
print(question_router.invoke({"question": question}))
end = time.time()
print(f"The time required to generate response by Router Chain in seconds:{end - start}")

In [ ]:
# Implement the retrieval_grader

import time
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.output_parsers import StrOutputParser

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing relevance
    of a retrieved document to a user question. If the document contains keywords related to the user question,
    grade it as relevant. It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question. \n
    Provide the binary score as a JSON with a single key 'score' and no premable or explaination.
     <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here is the retrieved document: \n\n {document} \n\n
    Here is the user question: {question} \n <|eot_id|><|start_header_id|>assistant<|end_header_id|>
    """,
    input_variables=["question", "document"],
)

# LLM with function call
llm = ChatOpenAI(model="gpt-4", temperature=0)

start = time.time()
retrieval_grader = prompt | llm | JsonOutputParser()

docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))
end = time.time()
print(f"The time required to generate response by the retrieval grader in seconds:{end - start}")

In [ ]:
## Implement the Generate Chain

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert for the COVID-19 fact-checking tasks.
    Based on pieces of retrieved context to detect the claim is true or false.
    You will have to give me the title and author of the context you refered in one sentence.
    Use three sentences maximum and keep the answer concise.
    If you don't know the answer, just say that you don't know. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# LLM
llm_g = ChatOpenAI(model_name="gpt-4", temperature=0)

# Chain
start = time.time()
rag_chain = prompt | llm_g | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

In [ ]:
# Implement the hallucination grader

# Prompt
prompt = PromptTemplate(
    template=""" <|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether
    an answer is grounded in / supported by a set of facts. Give a binary 'yes' or 'no' score to indicate
    whether the answer is grounded in / supported by a set of facts. Provide the binary score as a JSON with a
    single key 'score' and no preamble or explanation. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Here are the facts:
    \n ------- \n
    {documents}
    \n ------- \n
    Here is the answer: {generation}  <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "documents"],
)
start = time.time()
hallucination_grader = prompt | llm | JsonOutputParser()
hallucination_grader_response = hallucination_grader.invoke({"documents": docs, "generation": generation})

end = time.time()
print(f"The time required to generate response by the generation chain in seconds:{end - start}")
print(hallucination_grader_response)

In [ ]:
### Answer Grader

# Prompt
prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|> You are a grader assessing whether an
    answer is useful to resolve a question. Give a binary score 'yes' or 'no' to indicate whether the answer is
    useful to resolve a question. Provide the binary score as a JSON with a single key 'score' and no preamble or explanation.
     <|eot_id|><|start_header_id|>user<|end_header_id|> Here is the answer:
    \n ------- \n
    {generation}
    \n ------- \n
    Here is the question: {question} <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["generation", "question"],
)
start = time.time()
answer_grader = prompt | llm | JsonOutputParser()
answer_grader_response = answer_grader.invoke({"question": question,"generation": generation})
end = time.time()
print(f"The time required to generate response by the answer grader in seconds:{end - start}")
print(answer_grader_response)

In [ ]:
# Implement Web Search tool

import os
from langchain.utilities.tavily_search import TavilySearchAPIWrapper
from langchain_community.tools.tavily_search import TavilySearchResults
os.environ['TAVILY_API_KEY'] = "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"
#search = TavilySearchAPIWrapper()

web_search_tool = TavilySearchResults(k=3)

## Define the Graph State

In [ ]:
from typing_extensions import TypedDict
from typing import List

### State

class GraphState(TypedDict):
    question : str
    generation : str
    web_search : str
    documents : List[str]

In [ ]:
## Define nodes

from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    web_search = "No"
    for d in documents:
        score = retrieval_grader.invoke({"question": question, "document": d.page_content})
        grade = score['score']
        # Document relevant
        if grade.lower() == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        # Document not relevant
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            # We do not include the document in filtered_docs
            # We set a flag to indicate that we want to run web search
            web_search = "Yes"
            continue
    return {"documents": filtered_docs, "question": question, "web_search": web_search}
#
def web_search(state):
    """
    Web search based based on the question

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Appended web results to documents
    """

    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = web_search_tool.invoke({"query": question})
    web_results = "\n".join([d["content"] for d in docs])
    web_results = Document(page_content=web_results)
    if documents is not None:
        documents.append(web_results)
    else:
        documents = [web_results]
    return {"documents": documents, "question": question}
#

In [ ]:
## Define edges

def route_question(state):
    """
    Route question to web search or RAG.

    Args:
        state (dict): The current graph state

    Returns:
        str: Next node to call
    """

    print("---ROUTE QUESTION---")
    question = state["question"]
    print(question)
    source = question_router.invoke({"question": question})
    print(source)
    print(source['datasource'])
    if source['datasource'] == 'web_search':
        print("---ROUTE QUESTION TO WEB SEARCH---")
        return "websearch"
    elif source['datasource'] == 'vectorstore':
        print("---ROUTE QUESTION TO RAG---")
        return "vectorstore"
def decide_to_generate(state):
    """
    Determines whether to generate an answer, or add web search

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    web_search = state["web_search"]
    filtered_documents = state["documents"]

    if web_search == "Yes":
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print("---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, INCLUDE WEB SEARCH---")
        return "websearch"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [ ]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke({"documents": documents, "generation": generation})
    grade = score['score']

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question,"generation": generation})
        grade = score['score']
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [ ]:
## Add nodes

from langgraph.graph import END, StateGraph
workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("websearch", web_search) # web search
workflow.add_node("retrieve", retrieve) # retrieve
workflow.add_node("grade_documents", grade_documents) # grade documents
workflow.add_node("generate", generate) # generatae

In [ ]:
## Set the Entry Point and End Point

workflow.set_conditional_entry_point(
    route_question,
    {
        "websearch": "websearch",
        "vectorstore": "retrieve",
    },
)

workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "websearch": "websearch",
        "generate": "generate",
    },
)
workflow.add_edge("websearch", "generate")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "websearch",
    },
)

In [ ]:
## Compile the workflow

app = workflow.compile()

In [ ]:
## Test the workflow

from pprint import pprint
inputs = {"question": " Posts on social media claim the US Centers for Disease Control and Prevention (CDC) has revised down its death toll for COVID-19."}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

## Corrective RAG (CRAG) Performance

In [ ]:
questions = claim['question']
ground_truths = claim['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    inputs = {'question':query}
    try:
        for output in app.stream(inputs):
            for key, value in output.items():
                pprint(f"Finished running: {key}:")
        answers.append(value["generation"])
        docs = retriever.invoke(query)
        contexts.append([docs.page_content for docs in retriever.invoke(query)])
    except (RuntimeError, TypeError, NameError, AttributeError,KeyError):
        continue
    #contexts.append(docs[0].page_content)

In [ ]:
# To dict
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truths": ground_truths
}

In [ ]:
data['answer']

In [ ]:
# Convert dict to dataset
#dataset = Dataset.from_dict(data)
res = pd.DataFrame(data['answer'])
res

In [ ]:
res.to_csv('self_rag_g_data.csv')

In [ ]:
## A test
app = workflow.compile()

#
inputs = {"question": '''

14. Discoloration of fingers or toes is an uncommon but noted symptom of COVID-19.

'''}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
value["generation"]

## Self RAG

In [ ]:
### Retrieval Grader

from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_openai import ChatOpenAI


# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )


# LLM with function call
llm = ChatOpenAI(model="gpt-4", temperature=0)
structured_llm_grader = llm.with_structured_output(GradeDocuments)
# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader
question = claim['question'][0]
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

In [ ]:
## Implement the Generate Chain

prompt = PromptTemplate(
    template="""<|begin_of_text|><|start_header_id|>system<|end_header_id|>
    You are an expert for the COVID-19 fact-checking tasks.
    Based on pieces of retrieved context to detect the claim is true or false.
    You will have to give me the title and author of the context you refered in one sentence.
    Use three sentences maximum and keep the answer concise.
    If you don't know the answer, just say that you don't know. <|eot_id|><|start_header_id|>user<|end_header_id|>
    Question: {question}
    Context: {context}
    Answer: <|eot_id|><|start_header_id|>assistant<|end_header_id|>""",
    input_variables=["question", "document"],
)

# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# LLM
llm_g = ChatOpenAI(model_name="gpt-4", temperature=0)

# Chain
start = time.time()
rag_chain = prompt | llm_g | StrOutputParser()

# Run
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

In [ ]:
### Hallucination Grader


# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in generation answer."""

    binary_score: str = Field(
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )


# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n
     Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n {documents} \n\n LLM generation: {generation}"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader
hallucination_grader.invoke({"documents": docs, "generation": generation})

In [ ]:
### Answer Grader


# Data model
class GradeAnswer(BaseModel):
    """Binary score to assess answer addresses question."""

    binary_score: str = Field(
        description="Answer addresses the question, 'yes' or 'no'"
    )

# Prompt
system = """You are a grader assessing whether an answer addresses / resolves a question \n
     Give a binary score 'yes' or 'no'. Yes' means that the answer resolves the question."""
answer_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "User question: \n\n {question} \n\n LLM generation: {generation}"),
    ]
)

answer_grader = answer_prompt | structured_llm_grader
answer_grader.invoke({"question": question, "generation": generation})

In [ ]:
### Question Re-writer

# LLM
llm = ChatOpenAI(model="gpt-4", temperature=0)

# Prompt
system = """You a claim re-writer that converts an input claim to a better version that is optimized \n
     for vectorstore retrieval and fact-checking.
     Look at the input and try to reason about the underlying semantic intent / meaning."""
re_write_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        (
            "human",
            "Here is the initial claim: \n\n {question} \n Formulate an improved claim.",
        ),
    ]
)

question_rewriter = re_write_prompt | llm | StrOutputParser()
question_rewriter.invoke({"question": question})

## Graph State

In [ ]:
from typing_extensions import TypedDict
from typing import List


class GraphState(TypedDict):
    """
    Represents the state of our graph.

    Attributes:
        question: question
        generation: LLM generation
        documents: list of documents
    """

    question: str
    generation: str
    documents: List[str]

In [ ]:
## Define nodes

from langchain.schema import Document
def retrieve(state):
    """
    Retrieve documents from vectorstore

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, documents, that contains retrieved documents
    """
    print("---RETRIEVE---")
    question = state["question"]

    # Retrieval
    documents = retriever.invoke(question)
    return {"documents": documents, "question": question}
#
def generate(state):
    """
    Generate answer using RAG on retrieved documents

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): New key added to state, generation, that contains LLM generation
    """
    print("---GENERATE---")
    question = state["question"]
    documents = state["documents"]

    # RAG generation
    generation = rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question, "generation": generation}
#
def grade_documents(state):
    """
    Determines whether the retrieved documents are relevant to the question
    If any document is not relevant, we will set a flag to run web search

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Filtered out irrelevant documents and updated web_search state
    """

    print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
    question = state["question"]
    documents = state["documents"]

    # Score each doc
    filtered_docs = []
    for d in documents:
        score = retrieval_grader.invoke(
            {"question": question, "document": d.page_content}
        )
        grade = score.binary_score
        if grade == "yes":
            print("---GRADE: DOCUMENT RELEVANT---")
            filtered_docs.append(d)
        else:
            print("---GRADE: DOCUMENT NOT RELEVANT---")
            continue
    return {"documents": filtered_docs, "question": question}
#
def transform_query(state):
    """
    Transform the query to produce a better question.

    Args:
        state (dict): The current graph state

    Returns:
        state (dict): Updates question key with a re-phrased question
    """

    print("---TRANSFORM QUERY---")
    question = state["question"]
    documents = state["documents"]

    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

#

In [ ]:
## Define edges

def decide_to_generate(state):
    """
    Determines whether to generate an answer, or re-generate a question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Binary decision for next node to call
    """

    print("---ASSESS GRADED DOCUMENTS---")
    question = state["question"]
    filtered_documents = state["documents"]

    if not filtered_documents:
        # All documents have been filtered check_relevance
        # We will re-generate a new query
        print(
            "---DECISION: ALL DOCUMENTS ARE NOT RELEVANT TO QUESTION, TRANSFORM QUERY---"
        )
        return "transform_query"
    else:
        # We have relevant documents, so generate answer
        print("---DECISION: GENERATE---")
        return "generate"

In [ ]:
def grade_generation_v_documents_and_question(state):
    """
    Determines whether the generation is grounded in the document and answers question.

    Args:
        state (dict): The current graph state

    Returns:
        str: Decision for next node to call
    """

    print("---CHECK HALLUCINATIONS---")
    question = state["question"]
    documents = state["documents"]
    generation = state["generation"]

    score = hallucination_grader.invoke(
        {"documents": documents, "generation": generation}
    )
    grade = score.binary_score

    # Check hallucination
    if grade == "yes":
        print("---DECISION: GENERATION IS GROUNDED IN DOCUMENTS---")
        # Check question-answering
        print("---GRADE GENERATION vs QUESTION---")
        score = answer_grader.invoke({"question": question, "generation": generation})
        grade = score.binary_score
        if grade == "yes":
            print("---DECISION: GENERATION ADDRESSES QUESTION---")
            return "useful"
        else:
            print("---DECISION: GENERATION DOES NOT ADDRESS QUESTION---")
            return "not useful"
    else:
        pprint("---DECISION: GENERATION IS NOT GROUNDED IN DOCUMENTS, RE-TRY---")
        return "not supported"

In [ ]:
## Build Graph

from langgraph.graph import END, StateGraph

workflow = StateGraph(GraphState)

# Define the nodes
workflow.add_node("retrieve", retrieve)  # retrieve
workflow.add_node("grade_documents", grade_documents)  # grade documents
workflow.add_node("generate", generate)  # generatae
workflow.add_node("transform_query", transform_query)  # transform_query

# Build graph
workflow.set_entry_point("retrieve")
workflow.add_edge("retrieve", "grade_documents")
workflow.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "transform_query": "transform_query",
        "generate": "generate",
    },
)
workflow.add_edge("transform_query", "retrieve")
workflow.add_conditional_edges(
    "generate",
    grade_generation_v_documents_and_question,
    {
        "not supported": "generate",
        "useful": END,
        "not useful": "transform_query",
    },
)



In [ ]:
# Compile
app = workflow.compile()

## Self RAG Performance

In [ ]:
## Test the workflow

from pprint import pprint
inputs = {"question": '''

7. Effective ventilation of indoor spaces can reduce the airborne spread of COVID-19.

'''}
for output in app.stream(inputs):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])

value["generation"]

In [ ]:
data['answer']

In [ ]:
questions = claim['question']
ground_truths = claim['ground_truths']
contexts = []
answers = []

# Inference
for query in questions:
    inputs = {'question':query}
    try:
        for output in app.stream(inputs):
            for key, value in output.items():
                pprint(f"Finished running: {key}:")
        answers.append(value["generation"])
        docs = retriever.invoke(query)
        contexts.append([docs.page_content for docs in retriever.invoke(query)])
    except (RuntimeError, TypeError, NameError, AttributeError,KeyError):
        continue
    #contexts.append(docs[0].page_content)

In [ ]:
# To dict
data = {
    "question": questions,
    "answer": answers,
    'contexts': contexts,
    "ground_truths": ground_truths
}

In [ ]:
# Convert dict to dataset
#dataset = Dataset.from_dict(data)
res = pd.DataFrame(data['answer'])
res

In [ ]:
res.to_csv('self_rag_g_data.csv')

## Visualize the Agent / Graph

In [ ]:
print(app.get_graph().draw_mermaid())

In [ ]:
from langchain_core.runnables.graph import CurveStyle, NodeColors, MermaidDrawMethod
from IPython.display import display, HTML, Image

display(
    Image(
        app.get_graph().draw_mermaid_png(
            draw_method=MermaidDrawMethod.API,
        )
    )
)